### Define all packages and modules needed for the notebook.

In [2]:
import pandas as pd

### Read in the primary raw Dow Jones Industrial dataset.

In [3]:
stock_csv_path = "../DataCleaning/DowJones_Dirty.csv"
stock_df = pd.read_csv(stock_csv_path)

### Display sample of the raw Dow Jones Industrial dataset.
- Notably, we see that we have daily information since 1992.

In [4]:
stock_df.shape

(7327, 7)

In [5]:
stock_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1/2/1992,3152.100098,3172.629883,3139.310059,3172.399902,3172.399902,235500
1,1/3/1992,3172.399902,3210.639893,3165.919922,3201.500000,3201.500000,236200
2,1/6/1992,3201.500000,3213.330078,3191.860107,3200.100098,3200.100098,272800
3,1/7/1992,3200.100098,3210.199951,3184.479980,3204.800049,3204.800049,255100
4,1/8/1992,3204.800049,3229.199951,3185.820068,3203.899902,3203.899902,290400
...,...,...,...,...,...,...,...
7322,1/27/2021,30893.779300,30893.779300,30206.910160,30303.169920,30303.169920,5664700
7323,1/28/2021,30377.189450,30951.410160,30377.189450,30603.359380,30603.359380,4651400
7324,1/29/2021,30553.910160,30553.910160,29856.300780,29982.619140,29982.619140,5363200
7325,2/1/2021,30054.730470,30335.910160,30014.970700,30211.910160,30211.910160,3470600


### Adding column for 'Daily Change'
- We use the absolute value to account for changes regardless if decreasing or increasing.

In [9]:
stock_df["Daily Change"] = abs(stock_df["Close"]-stock_df["Open"])
stock_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Month,Day,Year
0,1/2/1992,3152.100098,3172.629883,3139.310059,3172.399902,3172.399902,235500,20.299804,1,2,1992
1,1/3/1992,3172.399902,3210.639893,3165.919922,3201.500000,3201.500000,236200,29.100098,1,3,1992
2,1/6/1992,3201.500000,3213.330078,3191.860107,3200.100098,3200.100098,272800,1.399902,1,6,1992
3,1/7/1992,3200.100098,3210.199951,3184.479980,3204.800049,3204.800049,255100,4.699951,1,7,1992
4,1/8/1992,3204.800049,3229.199951,3185.820068,3203.899902,3203.899902,290400,0.900147,1,8,1992


### Adding columns for 'Month', 'Day', and 'Year'
- Splitting the 'Date' column to be able to more freely identify months, days and years.
- Also allows us to have a joinable column against the NCVS dataset.

In [10]:
stock_df[['Month','Day','Year']] = stock_df['Date'].str.split('/',expand=True)
stock_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Month,Day,Year
0,1/2/1992,3152.100098,3172.629883,3139.310059,3172.399902,3172.399902,235500,20.299804,1,2,1992
1,1/3/1992,3172.399902,3210.639893,3165.919922,3201.500000,3201.500000,236200,29.100098,1,3,1992
2,1/6/1992,3201.500000,3213.330078,3191.860107,3200.100098,3200.100098,272800,1.399902,1,6,1992
3,1/7/1992,3200.100098,3210.199951,3184.479980,3204.800049,3204.800049,255100,4.699951,1,7,1992
4,1/8/1992,3204.800049,3229.199951,3185.820068,3203.899902,3203.899902,290400,0.900147,1,8,1992


### Create 'Year' groupby
- We will use the functions to identify specific values for each year.
- Specifically, we will identify yearly change, percent change, average close and average daily change.

In [11]:
group_stock = stock_df.groupby(["Year"])

year_open = group_stock["Open"].first()
year_close = group_stock["Close"].last()
year_change = year_close-year_open
perc_change = ((year_close/year_open) - 1)
avg_close = group_stock["Close"].mean()
avg_change = (group_stock["Daily Change"].mean()/avg_close)

### Create a new dataframe using the 'Year' groupby's
- Load in the variables (from the previous cell) as the dataframe columns.
- We can only use those groupby variables as they're indexed on 'Year'.

In [14]:
dowjones_df = pd.DataFrame({
    "First Open": year_open,
    "Last Close": year_close,
    "Average Close": avg_close,
    "Year Change": year_change,
    "Percent Change": perc_change,
    "Avg Daily Change/Volatility": avg_change
})
dowjones_df.head()

,First Open,Last Close,Average Close,Year Change,Percent Change,Avg Daily Change/Volatility
Year,,,,,,
1992,3152.100098,3301.110107,3284.080357,149.010009,0.047273,0.005008
1993,3301.100098,3754.090088,3524.918453,452.989990,0.137224,0.004080
1994,3754.100098,3834.439941,3794.222298,80.339843,0.021401,0.005150
1995,3834.399902,5117.120117,4494.276383,1282.720215,0.334530,0.004223
1996,5115.700195,6448.270020,5739.626469,1332.569825,0.260486,0.005727


### Reformat the 'dowjones_df'
- Adjust formatting for the aggregate variables.
- Insert the 'Year' index as a column (will help with typical dataframe features).

In [15]:
dowjones_df["Percent Change"] = dowjones_df["Percent Change"].map("{:.5f}".format)
dowjones_df["First Open"] = dowjones_df["First Open"].map("{:.2f}".format)
dowjones_df["Last Close"] = dowjones_df["Last Close"].map("{:.2f}".format)
dowjones_df["Average Close"] = dowjones_df["Average Close"].map("{:.2f}".format)
dowjones_df["Year Change"] = dowjones_df["Year Change"].map("{:.2f}".format)
dowjones_df["Avg Daily Change/Volatility"] = dowjones_df["Avg Daily Change/Volatility"].map("{:.5f}".format)
dowjones_df.reset_index(level=0, inplace=True)
dowjones_df

,Year,First Open,Last Close,Average Close,Year Change,Percent Change,Avg Daily Change/Volatility
0,1992,3152.10,3301.11,3284.08,149.01,0.04727,0.00501
1,1993,3301.10,3754.09,3524.92,452.99,0.13722,0.00408
2,1994,3754.10,3834.44,3794.22,80.34,0.02140,0.00515
3,1995,3834.40,5117.12,4494.28,1282.72,0.33453,0.00422
4,1996,5115.70,6448.27,5739.63,1332.57,0.26049,0.00573
5,1997,6447.50,7908.30,7447.01,1460.80,0.22657,0.00857
6,1998,7910.20,9181.43,8630.76,1271.23,0.16071,0.00884
7,1999,9184.01,11497.12,10481.56,2313.11,0.25186,0.00783
8,2000,11501.85,10787.99,10729.38,-713.86,-0.06206,0.00988
9,2001,10790.92,10021.57,10208.86,-769.35,-0.07130,0.00967


### Exporting the dataframe to be read into 'Analysis.ipynb'

In [6]:
dowjones_df.to_csv("DowJones_Clean")